In [1]:
import os
import sys

module_path = os.path.abspath("..")
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
from tensorflow.keras import backend as K
from interactions import ExactInteractions

import numpy as np


## Exact layer 

Creates a interaction matrix S were each entry $i$ and $j$ are defined by:

$s_{ij}=\begin{cases}1 & q_i = d_i\\0 & otherwise\end{cases}$

In [2]:
   
K.clear_session()

input_query = tf.keras.layers.Input((10,), dtype="int32")
input_sentence = tf.keras.layers.Input((12,), dtype="int32")

exact_interaction = ExactInteractions()

_out = exact_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()
    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
exact_interactions (ExactIntera (None, 10, 12)       0           input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


In [3]:
query = np.random.randint(0,4,(1, 10))
document = np.random.randint(0,4,(1, 12))

In [4]:
y = model.predict([query, document])
y

array([[[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [5]:
# add padding
query[:,5:] = np.zeros(5,)
document[:,6:] = np.zeros(6,)
y = model.predict([query, document])
y

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

# Semantic Layer

In [2]:
from interactions import SemanticInteractions

In [3]:
from nir.embeddings import FastText
from nir.tokenizers import Regex

cache_folder = "/backup/IR"
prefix_name = "disk4_5"

# load tokenizer
tk = Regex.load_from_json(cache_folder=cache_folder, prefix_name=prefix_name)

# load embedding matrix
ft = FastText.maybe_load(cache_folder = cache_folder,
                         prefix_name = prefix_name,
                         path = "/backup/pre-trained_embeddings/fasttext/wiki.en.bin",
                         tokenizer = tk)

emb_matrix = ft.embedding_matrix()

DEBUG created tokenizer disk4_5_RegexTokenizer
False False
[LOAD FROM CACHE] Load embedding matrix from /backup/IR/embedding_wiki_disk4_5_RegexTokenizer


In [4]:

K.clear_session()

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

semantic_interaction = SemanticInteractions(emb_matrix)

_out = semantic_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()
    

[EMBEDDING MATRIX SHAPE] (228107, 300)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
semantic_interactions (Semantic (None, 8, 4, 3)      600         input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 600
Trainable params: 600
Non-trainable params: 0
__________________________________________________________________________________________

In [5]:
query = np.random.randint(0,100000,(1, 8))
document = np.random.randint(0,100000,(1, 4))

In [6]:
y = model.predict([query, document])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[0.19258466 0.25187984 0.22666241 0.28524923]
 [0.24372493 0.11448298 0.21717232 0.29039758]
 [0.16095555 0.25080484 0.12638587 0.08130907]
 [0.11486951 0.19863784 0.13232115 0.18147907]
 [0.18183693 0.23730841 0.19377872 0.199918  ]
 [0.1356947  0.0835322  0.083666   0.15122163]
 [0.07151814 0.19953083 0.07166837 0.23351286]
 [0.01318797 0.15260708 0.22389746 0.11909546]]
[[ 0.05864156  0.05864156  0.05864156  0.05864156]
 [-0.05139646 -0.05139646 -0.05139646 -0.05139646]
 [-0.07237688 -0.07237688 -0.07237688 -0.07237688]
 [-0.11823501 -0.11823501 -0.11823501 -0.11823501]
 [-0.01284604 -0.01284604 -0.01284604 -0.01284604]
 [-0.16275871 -0.16275871 -0.16275871 -0.16275871]
 [ 0.0248914   0.0248914   0.0248914   0.0248914 ]
 [ 0.05576241  0.05576241  0.05576241  0.05576241]]
[[-0.00682486 -0.0156595   0.01011591 -0.03786472]
 [-0.00682486 -0.0156595   0.01011591 -0.03786472]
 [-0.00682486 -0.0156595   0.01011591 -0.03786472]
 [-0.00682486 -0.0156595   0.01011591 -0.03786472]
 [-0.00682

In [7]:
# add padding
query[:,4:] = np.zeros(4,)
document[:,2:] = np.zeros(2,)
y = model.predict([query, document])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[ 0.19258466  0.25187984  0.          0.        ]
 [ 0.24372493  0.11448298 -0.         -0.        ]
 [ 0.16095555  0.25080484 -0.         -0.        ]
 [ 0.11486951  0.19863784  0.          0.        ]
 [ 0.         -0.          0.          0.        ]
 [ 0.         -0.          0.          0.        ]
 [ 0.         -0.          0.          0.        ]
 [ 0.         -0.          0.          0.        ]]
[[ 0.05864156  0.05864156  0.          0.        ]
 [-0.05139646 -0.05139646 -0.         -0.        ]
 [-0.07237688 -0.07237688 -0.         -0.        ]
 [-0.11823501 -0.11823501 -0.         -0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[-0.00682486 -0.0156595   0.          0.        ]
 [-0.00682486 -0.0156595   0.          0.        ]
 [-0.00682486 -0.0156595   0.          0.        ]
 [-0.00682486 -0.0156595   0.